In [22]:
import numpy as np
import pandas as pd
import re

In [23]:
f = open('WhatsApp Chat.txt',encoding='utf-8')
data = f.read()

In [24]:
pattern = '\d{1,2}/\d{1,2}/\d{2},\s\d{1,2}:\d{2}\s'

messages = re.split(pattern, data)[1:]
dates = re.findall(pattern, data)

In [25]:
df = pd.DataFrame({'user_message': messages, 'message_date': dates})

In [26]:
df.head()

,user_message,message_date
0,AM - Shivam Singh Mtech Cse: Gaya\n,"8/9/21, 8:30"
1,AM - Shivam Singh Mtech Cse: Wo\n,"8/9/21, 8:30"
2,AM - Mrityunjay Mtech Cse: kaun sa na hua jo ...,"8/9/21, 8:31"
3,AM - Ankit Sharma Mtech Cse: I and j bracket\n,"8/9/21, 8:31"
4,AM - Shivam Singh Mtech Cse: Haan ho gaya\n,"8/9/21, 8:31"


In [27]:
df['user_message'][0:5]

0                  AM - Shivam Singh Mtech Cse: Gaya\n
1                    AM - Shivam Singh Mtech Cse: Wo\n
2    AM - Mrityunjay Mtech Cse: kaun  sa na hua jo ...
3       AM - Ankit Sharma Mtech Cse: I and j bracket\n
4          AM - Shivam Singh Mtech Cse: Haan ho gaya\n
Name: user_message, dtype: object

In [28]:
df['user_message'] = df['user_message'].map(lambda x: x.lstrip('PM - '))
df['user_message'] = df['user_message'].map(lambda x: x.lstrip('AM - '))

In [29]:
df.head()

,user_message,message_date
0,Shivam Singh Mtech Cse: Gaya\n,"8/9/21, 8:30"
1,Shivam Singh Mtech Cse: Wo\n,"8/9/21, 8:30"
2,rityunjay Mtech Cse: kaun sa na hua jo mai kru\n,"8/9/21, 8:31"
3,nkit Sharma Mtech Cse: I and j bracket\n,"8/9/21, 8:31"
4,Shivam Singh Mtech Cse: Haan ho gaya\n,"8/9/21, 8:31"


In [30]:
# convert message_date type
df['message_date'] = pd.to_datetime(df['message_date'], infer_datetime_format=True)
df['message_date'] = pd.to_datetime(df['message_date'], format='%m/%d/%y, %H:%M')
df.rename(columns={'message_date': 'date'}, inplace=True)

C:\Users\chokh\AppData\Local\Temp\ipykernel_36704\1386241987.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['message_date'] = pd.to_datetime(df['message_date'], infer_datetime_format=True)
C:\Users\chokh\AppData\Local\Temp\ipykernel_36704\1386241987.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['message_date'] = pd.to_datetime(df['message_date'], infer_datetime_format=True)


In [31]:
df.head(2)

,user_message,date
0,Shivam Singh Mtech Cse: Gaya\n,2021-08-09 08:30:00
1,Shivam Singh Mtech Cse: Wo\n,2021-08-09 08:30:00


In [32]:
users = []
messages = []
for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s', message)
    if entry[1:]:  # user name
        users.append(entry[1])
        messages.append(" ".join(entry[2:]))
    else:
        users.append('group_notification')
        messages.append(entry[0])

df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)

df['only_date'] = df['date'].dt.date
df['year'] = df['date'].dt.year
df['month_num'] = df['date'].dt.month
df['month'] = df['date'].dt.month_name()
df['day'] = df['date'].dt.day
df['day_name'] = df['date'].dt.day_name()
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute

period = []
for hour in df[['day_name', 'hour']]['hour']:
    if hour == 23:
        period.append(str(hour) + "-" + str('00'))
    elif hour == 0:
        period.append(str('00') + "-" + str(hour + 1))
    else:
        period.append(str(hour) + "-" + str(hour + 1))

df['period'] = period

In [33]:
df.head()

,date,user,message,only_date,year,month_num,month,day,day_name,hour,minute,period
0,2021-08-09 08:30:00,Shivam Singh Mtech Cse,Gaya\n,2021-08-09,2021,8,August,9,Monday,8,30,8-9
1,2021-08-09 08:30:00,Shivam Singh Mtech Cse,Wo\n,2021-08-09,2021,8,August,9,Monday,8,30,8-9
2,2021-08-09 08:31:00,rityunjay Mtech Cse,kaun sa na hua jo mai kru\n,2021-08-09,2021,8,August,9,Monday,8,31,8-9
3,2021-08-09 08:31:00,nkit Sharma Mtech Cse,I and j bracket\n,2021-08-09,2021,8,August,9,Monday,8,31,8-9
4,2021-08-09 08:31:00,Shivam Singh Mtech Cse,Haan ho gaya\n,2021-08-09,2021,8,August,9,Monday,8,31,8-9


## Code analyzing


In [34]:
df['user'][30:100]

30    Govind Narayan Mtech Cse
31         rityunjay Mtech Cse
32      Shivam Singh Mtech Cse
33                 Rajiv Kumar
34         rityunjay Mtech Cse
                ...           
95      Shivam Singh Mtech Cse
96    Govind Narayan Mtech Cse
97      Shivam Singh Mtech Cse
98      Shivam Singh Mtech Cse
99      Shivam Singh Mtech Cse
Name: user, Length: 70, dtype: object

In [35]:
# fetch the number of messages
num_messages = df.shape[0]
num_messages

39999

In [36]:
# fetch number of media messages
num_media_messages = df[df['message'] == '<Media omitted>\n'].shape[0]

In [37]:
num_media_messages

5672

In [39]:
# most busy user
x = df['user'].value_counts().head()

In [40]:
x

user
Shivam Singh Mtech Cse    13003
rityunjay Mtech Cse        6035
nkit Sharma Mtech Cse      5321
Ravish Mtech Cse           3385
Karamveer Mtech Cse        3041
Name: count, dtype: int64

In [41]:
df = round((df['user'].value_counts() / df.shape[0]) * 100, 2).reset_index().rename(
        columns={'index': 'name', 'user': 'percent'})

In [42]:
df

,percent,count
0,Shivam Singh Mtech Cse,32.51
1,rityunjay Mtech Cse,15.09
2,nkit Sharma Mtech Cse,13.30
3,Ravish Mtech Cse,8.46
4,Karamveer Mtech Cse,7.60
5,Govind Narayan Mtech Cse,6.39
6,Rishabh Mtech Cse,4.25
7,ranav Mtech Cse,2.84
8,Deepak Prajapat Mtech Cse,1.94
9,itranshu Mtech Cse,1.78
